In [16]:
import time

import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.nn import VGAE
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree, negative_sampling

from torch.utils.tensorboard import SummaryWriter

from gene_graph_dataset import G2GraphDataset
from phylognn_model import G3Median

In [2]:
gpuid = 0

train_p, test_p = 0.8, 0.2
train_batch, test_batch = 20, 8

In [3]:
device = torch.device('cuda:' + str(gpuid) if torch.cuda.is_available() else 'cpu')

In [4]:
dataset = G2GraphDataset('dataset_g2g', 1000, 10)

In [5]:
data_size = len(dataset)
train_size, test_size = (int)(data_size * train_p), (int)(data_size * test_p)

In [6]:
dataset = dataset.shuffle()
train_dataset = dataset[:train_size]
test_dataset = dataset[train_size:(train_size + test_size)]
# val_dataset = dataset[(train_size + test_size):]

In [7]:
train_loader = DataLoader(train_dataset, batch_size = train_batch, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = test_batch)
# val_loader = DataLoader(val_dataset, batch_size=8)

In [8]:
deg = torch.zeros(5, dtype=torch.long)
for data in train_dataset:
    d = degree(data.edge_index[1].type(torch.int64), 
               num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

In [9]:
# class EncoderNet(torch.nn.Module):
#     def __init__(self, in_channels, out_channels):
#         super(EncoderNet, self).__init__()

#         self.node_emb = Embedding(1000, 75)

#         aggregators = ['mean', 'min', 'max', 'std']
#         scalers = ['identity', 'amplification', 'attenuation']
        
#         self.conv_mu = GCNConv(75, out_channels)
#         self.conv_logstd = GCNConv(75, out_channels)
        
#         self.convs = ModuleList()
#         self.batch_norms = ModuleList()
#         for _ in range(4):
#             conv = PNAConv(in_channels=75, out_channels=75,
#                            aggregators=aggregators, scalers=scalers, deg=deg,
#                            # edge_dim=50, 
#                            towers=5, pre_layers=1, post_layers=1,
#                            divide_input=False)
#             # conv = GCNConv(in_channels=75, out_channels=75)
#             self.convs.append(conv)
#             self.batch_norms.append(BatchNorm(75))

#     def forward(self, x, edge_index): #, edge_attr, batch):
        
#         x = self.node_emb(x.squeeze())

#         for conv, batch_norm in zip(self.convs, self.batch_norms):
#             x = batch_norm(conv(x, edge_index)).relu()
#         return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [10]:
in_channels, out_channels = dataset.num_features, 16

In [11]:
model = VGAE(G3Median(in_channels, out_channels, deg)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10,
                              min_lr=0.00001)

In [12]:
writer = SummaryWriter(log_dir='runs/g2graph_batch')

In [13]:
def train(train_loader):
    model.train()
    
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        
        z = model.encode(data.x, data.edge_index)
        loss = model.recon_loss(z, data.pos_edge_label_index)
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    return total_loss/len(train_loader)

In [14]:
@torch.no_grad()
def test(test_loader):
    model.eval()
    loss, auc, ap = 0, 0, 0
    
    for data in test_loader:
        
        data = data.to(device)
        
        z = model.encode(data.x, data.edge_index)
        loss += model.recon_loss(z, data.pos_edge_label_index, 
                                 negative_sampling(data.pos_edge_label_index, 
                                                data.num_nodes,
                                                data.num_nodes*10))
                                 # data.neg_edge_label_index)
        tauc, tap = model.test(z, data.pos_edge_label_index, 
                               negative_sampling(data.pos_edge_label_index, 
                                                data.num_nodes,
                                                data.num_nodes*10))
                               # data.neg_edge_label_index)
        auc += tauc
        ap += tap
        
    return loss/len(test_loader), auc/len(test_loader), ap/len(test_loader)

In [ ]:
for epoch in range(1, 200 + 1):
    loss = train(train_loader)
    tloss, auc, ap = test(test_loader)
    
    scheduler.step(1 - auc)
    
    writer.add_scalar('Loss/train', loss, epoch)
    writer.add_scalar('Loss/test', tloss, epoch)
    writer.add_scalar('AUC/test', auc, epoch)
    writer.add_scalar('AP/test', ap, epoch)
    if epoch % 50 == 0:
        print(f'{time.ctime()} - '
              f'Epoch: {epoch:04d}, loss: {loss:.6f}, AUC: {auc:.6f}, '
              f'AP: {ap:.6f}, TL: {tloss:.6f}')

In [ ]:
writer.close()